In [1]:

import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from io import BytesIO
from ipywidgets import widgets
from tensorflow.keras.utils import load_img, img_to_array
import json
from sklearn.utils import resample
from PIL import Image


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:

# Folder dataset pakai yang shortcut, yang asli ada di gdrive machine learning
DATA_DIR = '/content/drive/MyDrive/Datasets'

# Folder output untuk menyimpan dataset YOLO, pakai ini dulu baru dipindah agar cepat
OUTPUT_DIR = '/content/yolo_dataset'
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'images'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels'), exist_ok=True)

# Mapping nominal ke kelas (YOLO format: 0, 1, 2, ...)
nominal_mapping = {
    '1000': 0,
    '2000': 1,
    '5000': 2,
    '10000': 3,
    '20000': 4,
    '50000': 5,
    '75000': 6,
    '100000': 7,
}

# Fungsi untuk membuat bounding box default karena dataset awal harus disesuaikan
def create_annotations(image_path, class_id, output_label_path):
    with Image.open(image_path) as img:
        width, height = img.size

        # Bounding box mencakup seluruh gambar
        x_center = 0.5
        y_center = 0.5
        bbox_width = 1.0
        bbox_height = 1.0

        # menyimpan anotasi dalam format YOLO
        with open(output_label_path, 'w') as f:
            f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")

for category in ['Genuine Rupiah Banknotes', 'Counterfeit Rupiah Banknotes']:
    category_path = os.path.join(DATA_DIR, category)
    is_genuine = 0 if 'Genuine' in category else 1  # 0 untuk asli, 1 untuk palsu

    for nominal, class_id in nominal_mapping.items():
        folder_path = os.path.join(category_path, nominal)
        if not os.path.exists(folder_path):
            continue

        for file_name in os.listdir(folder_path):
            if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                # Path input gambar
                image_path = os.path.join(folder_path, file_name)

                # Path output
                output_image_path = os.path.join(OUTPUT_DIR, 'images', f"{category}_{nominal}_{file_name}")
                output_label_path = os.path.join(OUTPUT_DIR, 'labels', f"{category}_{nominal}_{os.path.splitext(file_name)[0]}.txt")

                # menyalin gambar ke folder YOLO
                os.system(f"cp '{image_path}' '{output_image_path}'")

                # Buat anotasi bounding box
                combined_class_id = is_genuine * len(nominal_mapping) + class_id
                create_annotations(image_path, combined_class_id, output_label_path)


In [4]:
import shutil
# Copy folder yolo_dataset ke Google Drive, karena yang di colab hanya sementara
shutil.copytree('/content/yolo_dataset', '/content/drive/MyDrive/yolo_dataset_backup')

'/content/drive/MyDrive/yolo_dataset_backup'

In [5]:
yaml_content = """
train: /content/yolo_dataset/images/train
val: /content/yolo_dataset/images/val

# Jumlah kelas
nc: 16

# Nama kelas
names:
  - Genuine 1000
  - Genuine 2000
  - Genuine 5000
  - Genuine 10000
  - Genuine 20000
  - Genuine 50000
  - Genuine 75000
  - Genuine 100000
  - Counterfeit 1000
  - Counterfeit 2000
  - Counterfeit 5000
  - Counterfeit 10000
  - Counterfeit 20000
  - Counterfeit 50000
  - Counterfeit 75000
  - Counterfeit 100000
"""

# menyimpan file YAML
with open('/content/moneysense.yaml', 'w') as f:
    f.write(yaml_content)

print("File moneysense.yaml berhasil dibuat!")


File moneysense.yaml berhasil dibuat!


In [6]:
import shutil
shutil.move('/content/moneysense.yaml', '/content/drive/MyDrive/moneysense.yaml')


'/content/drive/MyDrive/moneysense.yaml'

In [7]:
# Lokasi file YAML di Google Drive
yaml_path = '/content/drive/MyDrive/moneysense.yaml'

# Membaca isi file
with open(yaml_path, 'r') as f:
    yaml_content = f.read()

# Menampilkan isi file
print(yaml_content)



train: /content/yolo_dataset/images/train
val: /content/yolo_dataset/images/val

# Jumlah kelas
nc: 16

# Nama kelas
names:
  - Genuine 1000
  - Genuine 2000
  - Genuine 5000
  - Genuine 10000
  - Genuine 20000
  - Genuine 50000
  - Genuine 75000
  - Genuine 100000
  - Counterfeit 1000
  - Counterfeit 2000
  - Counterfeit 5000
  - Counterfeit 10000
  - Counterfeit 20000
  - Counterfeit 50000
  - Counterfeit 75000
  - Counterfeit 100000



In [8]:
#memasang YOLOV5
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17063, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 17063 (delta 21), reused 27 (delta 12), pack-reused 17022 (from 1)
Receiving objects: 100% (17063/17063), 15.68 MiB | 12.80 MiB/s, done.
Resolving deltas: 100% (11716/11716), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.3/887.3 kB 16.1 MB/s eta 0:00:00


In [9]:
import random
import shutil
from glob import glob

# Lokasi dataset
image_dir = '/content/yolo_dataset/images'
label_dir = '/content/yolo_dataset/labels'

# Lokasi untuk train/val split
train_image_dir = '/content/yolo_dataset/images/train'
val_image_dir = '/content/yolo_dataset/images/val'
train_label_dir = '/content/yolo_dataset/labels/train'
val_label_dir = '/content/yolo_dataset/labels/val'

# Buat folder
os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

# Dapatkan semua file gambar
images = glob(os.path.join(image_dir, '*.jpg'))

# Split dataset
random.shuffle(images)
split_idx = int(0.8 * len(images))
train_images = images[:split_idx]
val_images = images[split_idx:]

# Salin gambar dan label ke folder train/val
for image_path in train_images:
    label_path = os.path.join(label_dir, os.path.basename(image_path).replace('.jpg', '.txt'))
    shutil.copy(image_path, train_image_dir)
    shutil.copy(label_path, train_label_dir)

for image_path in val_images:
    label_path = os.path.join(label_dir, os.path.basename(image_path).replace('.jpg', '.txt'))
    shutil.copy(image_path, val_image_dir)
    shutil.copy(label_path, val_label_dir)


In [10]:
!wandb offline

W&B offline. Running your script from this directory will only write metadata locally. Use wandb disabled to completely turn off W&B.


In [11]:
!python train.py --img 640 --batch 16 --epochs 50 --data /content/drive/MyDrive/moneysense.yaml --weights yolov5s.pt


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-11-24 15:17:58.692771: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 15:17:58.720531: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-24 15:17:58.728144: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to regi